In [88]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

In [89]:
database_name = 'telecom'
table_name = 'xdr_data'

connection_params = {"host": "localhost", "user": "postgres", "password": "Musy19", "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data'

df = pd.read_sql(sql_query, con= engine)

In [90]:
csv_file_path = "C:\\Users\\ADMIN\\10Academy\\Week-1\\telecom"
df.to_csv(csv_file_path, index=False)

In [91]:
df.columns = df.columns.str.replace(' ', '_')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer_Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start_ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End_ms                                    150000 non-null  float64
 5   Dur._(ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last_Location_Name                        148848 non-null  object 
 10  Avg_RTT_DL_(ms)     

In [93]:
#Separating numerical data and objects
num_col = df.select_dtypes (include = ['number']).columns
cat_col = df.select_dtypes(include=['object']).columns

df_num = df[num_col]
df_cat = df[cat_col]

In [94]:
application_columns = [
    
    'Social_Media_DL_(Bytes)','Social_Media_UL_(Bytes)','Youtube_DL_(Bytes)', 'Youtube_UL_(Bytes)','Netflix_DL_(Bytes)', 'Netflix_UL_(Bytes)','Google_DL_(Bytes)', 'Google_UL_(Bytes)','Email_DL_(Bytes)', 'Email_UL_(Bytes)','Gaming_DL_(Bytes)', 'Gaming_UL_(Bytes)','Other_DL_(Bytes)', 'Other_UL_(Bytes)'
    
    ]

In [95]:
# Handling missing values

if df_num.isnull().any().any():
    df_num = df_num.fillna(df_num.mean())

#Handling the Outliers
#Specifying the percentagge of values to be winsorized

winsorized_data = winsorize(df_num.to_numpy(), limits=[0.01, 0.01], axis=0)

#Convert winsorized_data back to the dataframe
df_num = pd.DataFrame(winsorized_data,columns=df_num.columns)

In [103]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 50 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer_Id                                 150001 non-null  float64
 1   Start_ms                                  150001 non-null  float64
 2   End_ms                                    150001 non-null  float64
 3   Dur._(ms)                                 150001 non-null  float64
 4   IMSI                                      150001 non-null  float64
 5   MSISDN/Number                             150001 non-null  float64
 6   IMEI                                      150001 non-null  float64
 7   Avg_RTT_DL_(ms)                           150001 non-null  float64
 8   Avg_RTT_UL_(ms)                           150001 non-null  float64
 9   Avg_Bearer_TP_DL_(kbps)                   150001 non-null  float64
 10  Avg_Bearer_TP_UL_(kb

In [104]:
#Standardizing Data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_num)

#Apply PCA
pca = PCA()
df_pca= pca.fit_transform(df_scaled)

#Visualization of variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = explained_variance_ratio.cumsum()

#Chooses the number of components based on explained variance
num_components = np.argmax(cumulative_explained_variance >= 0.95) + 1

pca = PCA(n_components = num_components)
df_pca = pca.fit_transform(df_scaled)


In [105]:
transposed_df = df_num.T
transposed_df

,0,1,2,3,4,5,6,7,8,9,...,149991,149992,149993,149994,149995,149996,149997,149998,149999,150000
Bearer_Id,1.311448e+19,1.311448e+19,1.311448e+19,1.311448e+19,1.311448e+19,1.311448e+19,1.311448e+19,1.304243e+19,1.311448e+19,1.304243e+19,...,7.349883e+18,1.311448e+19,1.311448e+19,1.311448e+19,1.304243e+19,7.277826e+18,7.349883e+18,1.311448e+19,1.311448e+19,1.013887e+19
Start_ms,7.700000e+02,2.350000e+02,1.000000e+01,4.860000e+02,5.650000e+02,4.390000e+02,6.120000e+02,5.920000e+02,1.210000e+02,1.000000e+01,...,7.940000e+02,1.140000e+02,7.900000e+01,8.300000e+01,6.150000e+02,4.510000e+02,4.830000e+02,2.830000e+02,6.960000e+02,4.991882e+02
End_ms,6.620000e+02,6.060000e+02,6.520000e+02,1.710000e+02,9.540000e+02,5.530000e+02,1.680000e+02,5.120000e+02,9.600000e+02,2.840000e+02,...,5.230000e+02,7.240000e+02,5.120000e+02,2.680000e+02,4.070000e+02,2.140000e+02,1.870000e+02,8.100000e+02,3.270000e+02,4.988009e+02
Dur._(ms),3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,3.987860e+05,...,6.166100e+04,8.494000e+04,1.256220e+05,1.135450e+05,5.958700e+04,8.123000e+04,9.797000e+04,9.824900e+04,9.791000e+04,1.046086e+05
IMSI,2.082014e+14,2.082019e+14,2.082003e+14,2.082014e+14,2.082014e+14,2.082014e+14,2.082014e+14,2.082010e+14,2.082014e+14,2.082003e+14,...,2.082017e+14,2.082014e+14,2.082014e+14,2.082003e+14,2.082014e+14,2.082022e+14,2.082019e+14,2.082017e+14,2.082021e+14,2.082016e+14
MSISDN/Number,3.366496e+10,3.368185e+10,3.376063e+10,3.375034e+10,3.369980e+10,3.366819e+10,3.366537e+10,3.376349e+10,3.369874e+10,3.365922e+10,...,3.376215e+10,3.376127e+10,3.362611e+10,3.364566e+10,3.366865e+10,3.365069e+10,3.366345e+10,3.362189e+10,3.361962e+10,3.378791e+10
IMEI,3.552121e+13,3.579401e+13,3.528151e+13,3.535661e+13,3.540701e+13,3.529841e+13,8.676270e+13,8.654640e+13,3.556241e+13,3.557311e+13,...,3.586061e+13,3.532701e+13,3.573531e+13,3.515551e+13,3.533251e+13,3.548311e+13,3.566051e+13,3.572121e+13,8.618620e+13,4.847455e+13
Avg_RTT_DL_(ms),4.200000e+01,6.500000e+01,1.097957e+02,1.097957e+02,1.097957e+02,1.097957e+02,1.020000e+02,3.900000e+01,1.097957e+02,9.700000e+01,...,2.700000e+01,3.700000e+01,4.600000e+01,1.097957e+02,3.130000e+02,3.200000e+01,2.700000e+01,4.300000e+01,3.700000e+01,1.097957e+02
Avg_RTT_UL_(ms),5.000000e+00,5.000000e+00,1.766288e+01,1.766288e+01,1.766288e+01,1.766288e+01,5.000000e+00,1.800000e+01,1.766288e+01,7.000000e+00,...,2.000000e+00,4.000000e+00,6.000000e+00,1.766288e+01,9.000000e+00,0.000000e+00,2.000000e+00,6.000000e+00,5.000000e+00,1.766288e+01
Avg_Bearer_TP_DL_(kbps),2.300000e+01,1.600000e+01,6.000000e+00,4.400000e+01,6.000000e+00,7.000000e+01,2.200000e+01,3.698000e+03,4.600000e+01,3.845000e+03,...,6.200000e+01,2.300000e+01,4.300000e+01,5.500000e+01,6.342000e+04,5.200000e+01,2.300000e+01,4.300000e+01,3.400000e+01,1.330005e+04


In [113]:

sessions_per_user = df_num.groupby('MSISDN/Number')['Bearer_Id'].count().reset_index()
user_session_duration = df_num.groupby('MSISDN/Number')['Dur._(ms)'].sum().reset_index()

session_frequency = sessions_per_user / user_session_duration
#session_frequency.columns = ['MSISDN/Number', 'session_frequency']

print(session_frequency)


        Bearer_Id  Dur._(ms)  MSISDN/Number
0             NaN        NaN            1.0
1             NaN        NaN            1.0
2             NaN        NaN            1.0
3             NaN        NaN            1.0
4             NaN        NaN            1.0
...           ...        ...            ...
105412        NaN        NaN            1.0
105413        NaN        NaN            1.0
105414        NaN        NaN            1.0
105415        NaN        NaN            1.0
105416        NaN        NaN            1.0

[105417 rows x 3 columns]


In [98]:
#Tracking users engagement using duration of session
session_duration = df_num[['MSISDN/Number', 'Dur._(ms)']]

session_duration['Total_Duration_Sec'] = session_duration['Dur._(ms)'] / 1000

session_duration['Avg_Duration_Sec'] = session_duration.groupby('MSISDN/Number')['Total_Duration_Sec'].transform('mean')

session_duration.head(10)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16196\1082717445.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session_duration['Total_Duration_Sec'] = session_duration['Dur._(ms)'] / 1000
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16196\1082717445.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session_duration['Avg_Duration_Sec'] = session_duration.groupby('MSISDN/Number')['Total_Duration_Sec'].transform('mean')


,MSISDN/Number,Dur._(ms),Total_Duration_Sec,Avg_Duration_Sec
0,3.366496e+10,398786.0,398.786,329.17150
1,3.368185e+10,398786.0,398.786,365.92450
2,3.376063e+10,398786.0,398.786,398.78600
3,3.375034e+10,398786.0,398.786,398.78600
4,3.369980e+10,398786.0,398.786,398.78600
5,3.366819e+10,398786.0,398.786,191.19300
6,3.366537e+10,398786.0,398.786,398.78600
7,3.376349e+10,398786.0,398.786,393.66750
8,3.369874e+10,398786.0,398.786,398.78600
9,3.365922e+10,398786.0,398.786,359.73775


In [99]:

user_traffic = df_num[['MSISDN/Number', 'Total_DL_(Bytes)', 'Total_UL_(Bytes)']]

# Calculate total traffic per user (in Bytes)
user_traffic['Total_Traffic_Bytes'] = user_traffic['Total_DL_(Bytes)'] + user_traffic['Total_UL_(Bytes)']

# Calculate average traffic per user (optional)
user_traffic['Avg_Traffic_Bytes'] = user_traffic.groupby('MSISDN/Number')['Total_Traffic_Bytes'].transform('mean')

user_traffic.head(10)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16196\3713542376.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_traffic['Total_Traffic_Bytes'] = user_traffic['Total_DL_(Bytes)'] + user_traffic['Total_UL_(Bytes)']
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_16196\3713542376.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_traffic['Avg_Traffic_Bytes'] = user_traffic.groupby('MSISDN/Number')['Total_Traffic_Bytes'].transform('mean')


,MSISDN/Number,Total_DL_(Bytes),Total_UL_(Bytes),Total_Traffic_Bytes,Avg_Traffic_Bytes
0,3.366496e+10,308879636.0,36749741.0,345629377.0,3.130749e+08
1,3.368185e+10,653384965.0,53800391.0,707185356.0,4.654754e+08
2,3.376063e+10,279807335.0,27883638.0,307690973.0,3.076910e+08
3,3.375034e+10,846028530.0,43324218.0,889352748.0,8.893527e+08
4,3.369980e+10,569138589.0,38542814.0,607681403.0,6.076814e+08
5,3.366819e+10,754452212.0,30307754.0,784759966.0,5.340458e+08
6,3.366537e+10,70562047.0,47925246.0,118487293.0,1.184873e+08
7,3.376349e+10,775350343.0,58813016.0,834163359.0,5.792480e+08
8,3.369874e+10,861612261.0,42363146.0,903975407.0,9.039754e+08
9,3.365922e+10,850570347.0,22417975.0,872988322.0,5.809985e+08


In [100]:
user_traffic.locn['Total_Traffic_Bytes'] = user_traffic['Total_DL_(Bytes)'] + user_traffic['Total_UL_(Bytes)']
user_traffic['Total_Traffic_Bytes'] 

AttributeError: 'DataFrame' object has no attribute 'locn'

In [ ]:
# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
user_traffic_imputed = imputer.fit_transform(user_traffic[['Total_Traffic_Bytes', 'Avg_Traffic_Bytes']])

# Normalize the engagement metrics
scaler = StandardScaler()
normalized_data = scaler.fit_transform(user_traffic_imputed)

# Run k-means clustering (k=3)
kmeans = KMeans(n_clusters=3, random_state=42)
df_num['Cluster'] = kmeans.fit_predict(normalized_data)


# Plotting the clusters
plt.figure(figsize=(10, 6))
plt.scatter(df_num['Total_Traffic_Bytes'], df_num['Avg_Traffic_Bytes'], c=df_num['Cluster'], cmap='viridis')
plt.title('K-Means Clustering of User Engagement')
plt.xlabel('Total Traffic (Bytes)')
plt.ylabel('Average Traffic (Bytes)')
plt.show()